In [3]:
import torch
import transformers

print("Torch version:{}".format(torch.__version__)) # Torch version:1.12.1
print("Cuda version: {}".format(torch.version.cuda)) # Cuda version: 11.3
print("transformers version: {}".format(transformers.__version__)) # transformers 4.28.0
print("GPU 사용 가능여부: {}".format(torch.cuda.is_available()))

Torch version:1.12.1
Cuda version: 11.3
transformers version: 4.28.0
GPU 사용 가능여부: True


## 1. BaseModel and Dataset for RLHF

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import numpy

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### 1-1. kogpt-2의 tokenize 과정

In [5]:
tokenizer.max_model_input_sizes

{'gpt2': 1024,
 'gpt2-medium': 1024,
 'gpt2-large': 1024,
 'gpt2-xl': 1024,
 'distilgpt2': 1024}

In [6]:
input_txt = "바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까."

In [7]:
tokens = tokenizer(input_txt).tokens()
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].numpy()

In [8]:
pd.options.display.max_columns = 40
pd.options.display.max_rows = 60
df = pd.DataFrame([tokens, input_ids[0]], index=["kogpt-2_tokens", "Input_IDs"])
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
kogpt-2_tokens,▁바람,도,▁없는,▁공중에,▁수직,의,▁파,문을,▁내,이며,▁고,요,히,▁떨어지는,▁오동,잎은,▁누,구의,▁발자,취,▁입,니까,.
Input_IDs,10891,7235,9712,49207,14438,8143,9203,9941,9094,9639,9065,8084,8811,21215,34769,19985,9669,10139,21626,8408,9241,23775,389


#### Greedy Search Decoding

In [9]:
max_length=128
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, 
                               max_length=max_length, 
                               do_sample=False)

print(tokenizer.decode(output_greedy[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까.'
"그렇다면 그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리


- 시퀀스가 반복되어 출력되는 것을 볼 수 있다.
- 이는 **Greedy Search Decoding**시 발견되는 전형적인 현상이다.

#### Beam-Search Decoding + n-gram penalty

In [10]:
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_beam = model.generate(input_ids, 
                             max_length=max_length, 
                             num_beams=10, 
                             no_repeat_ngram_size=2,
                             do_sample=False)

print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까.'
"그렇지 않습니다."
"어떻게 된 일입니까?"
그녀는 고개를 갸웃거렸다.
"아니, 그게 무슨 말씀이신지 모르겠습니다만."
"무슨 말씀인지 알 수가 없군요."
아무런 대답도 하지 않은 채 그녀는 고개를 끄덕였다.
"그래, 알았어."
그녀의 눈에서 눈물이 주르륵 흘러내렸다.
그녀가 다시 입을 열었다.
"정말 죄송합니다, 고마워요, 고맙습니다"
"


- 입력 시퀀스와 상관 없어 보이는 긴 문단이 생성된다.
- 문장 간의 정합성과 일관성이 떨어지는 모습을 볼 수 있다.

#### Beam-Search Decoding + n-gram penalty + Sampling

In [11]:
output_beam = model.generate(input_ids, 
                             max_length=max_length, 
                             num_beams=7, 
                             no_repeat_ngram_size=2,
                             do_sample=True, 
                             temperature=2.0, top_k=50)

print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까."
"우습지. 우습기는 한 번 더 말해봐."
"아무튼, 아까 얘기하고 싶은 것 있으세요?"
아이는 고개를 끄덕였다.
"이런, 오늘은 내 얘기를 해 봐."
그러자 아이는 한숨을 푹 내쉬었다.
"다음에 뵙겠어요."
아이도 그렇게 말하면서 눈을 흘겼다.
"그럼, 우리 얘기는 뭐가 됐는지 한번 들어봐요. 이따가 꼭."
그날은 아침 7시에 일어났다.



generate 함수의 인자인 temperature와 top_k는 텍스트 생성 결과에 다양성을 부여하기 위해 사용되는 매개변수들입니다.

1. **temperature**
   - 모델의 출력 분포에 대한 확률을 조정하는 역할을 합니다.
   - 0보다 큰 값으로, 값이 클수록 출력 분포가 더 균등해지고 다양성이 증가합니다.
   - 너무 낮은 값은 가장 높은 확률의 토큰만 생성하게 되어 창의성이 부족해집니다.
   - 너무 높은 값은 출력이 지나치게 무작위해져 의미 없는 결과가 나올 수 있습니다.

2. **top_k**
   - 상위 k개의 확률 높은 토큰들만 샘플링 대상으로 고려합니다. 
   - 이 값이 작을수록 더 예측 가능한 텍스트가 생성됩니다.
   - 값이 클수록 덜 예측 가능하지만 더 다양한 텍스트가 생성될 수 있습니다.
   - 0으로 설정하면 top-k 필터링이 비활성화됩니다.

따라서 temperature와 top_k를 적절히 조절하여 생성 결과의 다양성과 예측 가능성 사이의 균형을 맞출 수 있습니다.

- 높은 temperature, 높은 top_k: 매우 다양하지만 얼토당토않은 결과 생성 가능
- 낮은 temperature, 낮은 top_k: 매우 예측 가능하고 일관된 결과 생성 

#### Top_p Sampling

In [12]:
output_beam = model.generate(input_ids, 
                             max_length=max_length, 
                             num_beams=7, 
                             no_repeat_ngram_size=2,
                             do_sample=True, 
                             top_p=0.90)

print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까.'
"아니, 이게 무슨 일입니까?"
"그렇습니다, 아저씨."
"어떻게 된 거죠? 저도 모르는 사이에 저를 찾아온 거예요."
아저씨는 고개를 끄덕였다.
"무슨 일이에요? 어디서 뭘 하고 있는 겁니까, 아줌마."
아줌마는 어깨를 으쓱했다.
"누구에게 무슨 짓을 했는지 알겠어요. 저는 뭔가 잘못한 게 있다고 생각했습니다."
그녀는 고개를 저었다.



#### Top_p에 대한 설명

Top_p는 텍스트 생성 시 사용되는 중요한 매개변수 중 하나입니다. 이 매개변수는 텍스트 생성 모델이 다음 단어를 선택할 때, 가능한 단어 후보를 제한하는 데 사용됩니다. 구체적으로는 누적 확률 질량(cumulative probability mass)을 기준으로 상위 토큰들을 선택합니다.

##### 동작 방식
1. 모델의 마지막 출력에 대한 확률 분포를 계산합니다.
2. 확률 분포를 내림차순으로 정렬합니다.
3. Top_p 값에 해당하는 누적 확률 질량에 포함되는 상위 토큰들만을 샘플링 후보로 선택합니다.
4. 남은 토큰들의 확률을 0으로 설정합니다.
5. 이렇게 필터링된 분포에서 다음 단어를 샘플링합니다.

예를 들어, top_p=0.9로 설정한다면, 누적 확률이 90%에 해당하는 지점까지의 상위 토큰들만이 샘플링 후보로 고려됩니다.

Top_p는 top_k와 유사한 역할을 하지만, 특정 개수의 토큰이 아닌 누적 확률 질량을 기준으로 삼는다는 점에서 차이가 있습니다.

##### 성능
Top_p는 일반적으로 top_k보다 성능이 우수하다고 알려져 있습니다. 이는 top_p가 확률 분포의 형태를 더 잘 반영하기 때문입니다.

##### 활용
Top_p는 생성된 텍스트의 다양성과 품질을 제어하기 위해 사용됩니다. Top_k 또는 temperature와 함께 적절히 조정하여 원하는 출력을 얻을 수 있습니다.


In [13]:
input_txt = '너가 나 좋아한다고 했잖아. 왜 갑자기 맘이 바뀌는데?'

tokens = tokenizer(input_txt)["input_ids"]

input_ids = torch.tensor(tokens).unsqueeze(0) # 배치 차원 추가

model = model.to('cuda')
input_ids = input_ids.to('cuda')

# 모델 연산
output = model(input_ids)

output_beam = model.generate(input_ids, 
                             max_length=max_length,
                             num_beams=7,
                             no_repeat_ngram_size=2,
                             do_sample=True,
                             top_p=0.90)

print(tokenizer.decode(output_beam[0], skip_special_tokens=True))

너가 나 좋아한다고 했잖아. 왜 갑자기 맘이 바뀌는데?"
"그런데 그게 무슨 소리예요? 무슨 일이에요!"
그녀는 고개를 끄덕였다.
"아니야. 괜찮아."
"뭐라고 했어? 왜 그러는 거야? 아빠가 왜 그렇게 화를 내냐고."
그녀의 눈에서 눈물이 주르르 흘러내렸다.
"엄마! 엄마가 왜 이렇게 화를 내는지 알았어요. 엄마한테 왜 화를 낸 거예요. 왜 그런 말을 했는지 알 수가 없어요."
아빠는 눈물을 흘리며 고개를 떨구었다.
"


#### RM에 사용할 데이터셋

In [14]:
import json

data_path_2_RM = './KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl'
with open(data_path_2_RM, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

10220


[{'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?',
  'completion_0': 'Allow me to answer your question. I know that you are curious about me.',
  'completion_1': '번디는 다양한 인터뷰자들과 뉴스홍보 담당자들과의 면담 때 밝혔다.',
  'completion_2': '라이언에게 말했다.',
  'ranking': [2, 1, 0]},
 {'prompt': '개포주공아파트는 몇 단지로 이루어져 있나?',
  'completion_0': '개포주공아파트는 다섯 단지로 이루어져 있습니다.',
  'completion_1': '이날 목송에서 구글상위노',
  'completion_2': '개포주공아파트는 총 27개 단지로 이루어져 있습니다.',
  'ranking': [2, 0, 1]},
 {'prompt': '김영삼의 후보 시절 지역표심을 겨냥한 발언을 문제삼은 후보는?',
  'completion_0': 'The diameter of the Metallic domain is bigger than the Hyperonic domain.',
  'completion_1': '이 질문은 조금 불분명합니다. 김영삼 대통령이 후보 시절에 어떤 발언을 했고, 누가 그 발언을 문제삼았는지에 따라 답이 다를 수 있습니다.\\n\\n만약 김영삼 대통령이 후보 시절에 지역표심을 겨냥한 발언을 했다는 가정하에, 그 발언을 문제삼은 후보가 누구였는지를 대답하자면, 그 답은 이화선 당시 민주당 대통령 후보가 될 것입니다. 1992년 총선 때, 김영삼 대선후보는 "집값이 오른 노량진역 부근의 부동산 가격은 세월호 폭침 후 \\\'강남 도시재생\\\' 일환으로 상승했다"는 발언을 했습니다. 하지만 이화선 후보는 이 발언을 "전국적으로 경제적 발전이 이루어지지 않은 지방민의 마음을 멀리해지려는 무례한 발언"이라고 비판하며 문

#### PPO 학습에 사용할 데이터셋

In [15]:
data_path_3_PPO = './KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl'
with open(data_path_3_PPO, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

12000


[{'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?'},
 {'prompt': '개포주공아파트는 몇 단지로 이루어져 있나?'},
 {'prompt': '김영삼의 후보 시절 지역표심을 겨냥한 발언을 문제삼은 후보는?'}]

## 2. Supervised Fine-Tuning

### SFT
- kogpt-2를 instruction dataset으로 SFT를 진행

In [16]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim import Adam
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments
from copy import deepcopy
import copy
import logging
import json
from dataclasses import dataclass

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


- 모델과 Tokenizer 불러오기

In [17]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

print(tokenizer)

GPT2TokenizerFast(name_or_path='skt/kogpt2-base-v2', vocab_size=51200, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=True)


In [18]:
from typing import Optional, Dict, Sequence

class SFT_dataset(Dataset):

    def __init__(self, data_path_1_SFT: str, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")

        pattern_instruction = 'prompt'  # instruction
        pattern_output = 'completion'  # response

        with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
            list_data_dict = json.load(json_file)

        PROMPT_DICT = {
            "prompt_input": (
                "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
            )
        }

        prompt_input = PROMPT_DICT["prompt_input"]

        sources = []
        for example in list_data_dict:
            tmp = prompt_input.format_map(example)
            sources.append(tmp)

        targets = []
        for example in list_data_dict:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")
        examples = [s + t for s, t in zip(sources, targets)]

        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target

        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = -100

        data_dict = dict(input_ids=input_ids, labels=labels)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))


    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )


    def __len__(self):
        return len(self.input_ids)


    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

In [19]:
@dataclass
class DataCollatorForSupervisedDataset(object): 

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value= -100)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [20]:
train_dataset = SFT_dataset(data_path_1_SFT='./KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl', tokenizer=tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

print('input : %s'%train_dataset.input_ids[0])
print('output: %s'%train_dataset.labels[0])

input : tensor([  739,   378,   378,   378, 14659, 13394, 37091, 10651,   383, 25841,
         8006, 14914,   375,  7673, 20479,  8091, 22311,  9036, 30902, 13675,
          375,   378,   378,   378, 41951,   454,  9549, 20549,   383,  8142,
         7192, 14914,   382, 37767, 13753,  8263,  7166,   739,  8352,  7659,
         9594, 25585, 13600,  8022,  9378, 11532,  9887, 11218,  9111, 16691,
        10351, 10561,  9128, 20479,  8091,  9065,  9446,  9036, 28420, 26521,
        10163, 26367,  6958,  9030,  9882, 12317, 25882,  9209, 37194, 10351,
         9036, 12168, 10529, 15989,  9719, 15434, 10552, 11188, 13362,  9036,
        15805, 11300, 11846,  9146, 16691,  9181,  7397, 15806, 13480, 11342,
        17596,  9161, 19996,  9025, 25006, 18595,  9966, 12592, 10751, 11814,
         8711,  9046, 12450,  9117,  7377, 12521,     1])
output: tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -10

In [21]:
# train_dataset.input_ids[0]를 리스트로 변환하여 2차원으로 만들기
original_tensor = train_dataset.input_ids[0]

# 원하는 shape으로 reshape
reshaped_tensor = original_tensor.view(1, -1)

output_beam_Q11 = model.generate(reshaped_tensor, max_length=max_length, num_beams=7, no_repeat_ngram_size=2,
                             do_sample=True, top_p=0.90)
print(tokenizer.decode(output_beam_Q11[0]))

### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.</s><unk><unk> <unk>koreafoods.co.kr.
#koreanbulld


### 모델 정의 및 학습

In [22]:
training_args = TrainingArguments(
    output_dir="/aiffel/KoChatGPT/test",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=5,
    prediction_loss_only=True,
    fp16 = True
    )
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

In [23]:
import gc

torch.cuda.empty_cache()
gc.collect()

0

In [24]:
trainer.train()
model.save_pretrained('/aiffel/KoChatGPT/output_1_SFT')

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,3.149500
1000,2.947900
1500,2.875000
2000,2.759300
2500,2.742400
3000,2.656700


In [25]:
generator = pipeline('text-generation', model='/aiffel/KoChatGPT/output_1_SFT', tokenizer=tokenizer)

generation_args = dict(   
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375, # \n   
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = ['불고기용 고기 한우에요?',
               '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
               '시카고 오헤어 국제공항은 어디에 있어?',
               '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

list_result = generator(list_prompt, **generation_args)   
for prompt, result in zip(list_prompt, list_result):
    print()
    print((result[0]['generated_text']))

/opt/conda/lib/python3.9/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(



### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 인공지능 어시스턴트이기 때문에 고기를 먹을 수 없습니다. 하지만 일반적으로 불고기는 건강에 좋은 식품 중 하나입니다. 쇠고기의 고기는 단백질, 지방, 철분, 칼슘, 철분, 인 등이 풍부하기 때문에 건강에 좋습니다. 또한, 소고기와 돼지고기를 섞어 만든 양념도 인기 있는 요리 중 하나입니다.

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):'리처드 닉슨은 42대 부통령직을 수행했습니다. 리처드 닉슨은 46대 부통령직을 맡았습니다. 리처드 닉슨은 48대 부통령을 역임하였습니다. 리처드는 40대 부통령을 맡았던 적이 있습니다. 리처드는 47대 부통령을 맡은 적이 없습니다. 리처드의 재임 기간 동안, 리처드 닉슨은

### Instruction(명령어):
시카고 오헤어 국제공항은 어디에 있어?

### Response(응답):'시카고 오 헤어 국제공항은 미국 캘리포니아주 샌프란시스코에 위치해 있습니다. American International Pacific Language model, Translation of the Korean Capilities in Canada Orientality and Distributed Commissions.

### Instruction(명령어):
오늘 미세먼지 어때?

### Response(응답):'저는 인공지능 챗봇이므로 미세먼지 정보를 알 수 없습니다. 하지만, 미세먼지 예보나 예보 사이트를 통해 미세먼지 농도를 확인하실 수 있으니 참고하시기 바랍니다. 감사합니다. Please provide model, I do not have
